In [2]:

# Download Data from Yahoo Finance
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Pour lire les données de Yahoo Finance, vous devez installer le toolbox yfinance
import yfinance as yf 

def get_dow():
    start_sp = dt.datetime(2004,12,31)
    end_sp = dt.datetime(2019, 12,31)

    # Dow Jones index
    # 1st Dow Jones Index + 30 components
    dow30 = ['^DJI','AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'XOM',
            'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
            'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']

    dow_data = yf.download(dow30, start=start_sp, end=end_sp,group_by="ticker")

    return dow_data.transpose()

In [24]:
#TODO: ne pas ignorer tout les erreurs
def calculate_fivedays_roi(stock_price, date):
    roi = None
    try:
        past_date = date - np.timedelta64(5, "D")
        current_value = stock_price.loc[date]
        past_value = stock_price.loc[past_date]
        roi = (current_value - past_value)/past_value * 100        
    except KeyError:
        pass        
    return roi

# Calculate roi over a period of time
def get_rois(stock_prices, ticker, start_date, end_date):
    rois = []
    dates = []
    company_stock_price = stock_prices.loc[(ticker, "Close")]

    # calculate de ROI over the last 250 days
    curr_date = start_date
    while(curr_date < end_date):
        roi = calculate_fivedays_roi(company_stock_price, curr_date)
        if roi:
            rois.append(roi)
            dates.append(curr_date)
        curr_date = curr_date + np.timedelta64(1, "D")                

    return pd.Series(rois,index=dates)

# Calculate roi for a bunch of campanies
def get_companies_rois(stock_prices, start_date, end_date):
    tickers = stock_prices.index.get_level_values(0).unique().array
            
    result = pd.DataFrame()
    for ticker in tickers:             
        result[ticker] = get_rois(stock_prices, ticker, start_date, end_date)
    
    return result
    
def calculate_mean_roi(rois):
    rois["Mean"] = rois.mean(axis=1)
    return rois

In [28]:
def calculate_var(stock_prices):
    period = 250
    five_percent = int(period * 0.05)
    # [min date + 5, higher date] (we need 5 days buffer to calculate roi)
    start_date = dow_prices.columns.array.min() + np.timedelta64( 5, "D")
    end_date = dow_prices.columns.array.max()
    rois = get_companies_rois(stock_prices, start_date, end_date)
    rois = calculate_mean_roi(rois)
    
    var_array = []
    curr_date = start_date + np.timedelta64( 250, "D")
    while(curr_date < end_date):
        period_end = curr_date
        period_begin = period_end - np.timedelta64(period, "D")
        var = rois.loc[period_begin:period_end]["Mean"].nsmallest(five_percent)[five_percent-1]
        var_array.append(var)
        curr_date = curr_date + np.timedelta64(1, "D")
    return var_array

In [29]:
dow_prices = get_dow()
var_array = calculate_var(dow_prices)
var_array

[*********************100%***********************]  31 of 31 completed


[-1.1559050875519452,
 -0.9907647324441332,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.1559050875519452,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 -1.185691824934755,
 

In [17]:
a.loc[np.datetime64("2005-09-12"):np.datetime64("2005-09-16")]["CSCO"].nlargest(2)

2005-09-12    0.434782
2005-09-13   -0.653244
Name: CSCO, dtype: float64

In [13]:
dow_prices

Date              2004-12-31    2005-01-03    2005-01-04    2005-01-05  \
CSCO Open       1.939000e+01  1.942000e+01  1.938000e+01  1.847000e+01   
     High       1.950000e+01  1.961000e+01  1.939000e+01  1.888000e+01   
     Low        1.926000e+01  1.927000e+01  1.854000e+01  1.845000e+01   
     Close      1.932000e+01  1.932000e+01  1.856000e+01  1.857000e+01   
     Adj Close  1.495589e+01  1.495589e+01  1.436756e+01  1.437530e+01   
...                      ...           ...           ...           ...   
TRV  High       3.750000e+01  3.786000e+01  3.773000e+01  3.713000e+01   
     Low        3.707000e+01  3.725000e+01  3.697000e+01  3.680000e+01   
     Close      3.707000e+01  3.725000e+01  3.704000e+01  3.698000e+01   
     Adj Close  2.571363e+01  2.583849e+01  2.569282e+01  2.565120e+01   
     Volume     1.745300e+06  2.765600e+06  1.776600e+06  2.447200e+06   

Date              2005-01-06    2005-01-07    2005-01-10    2005-01-11  \
CSCO Open       1.867000e+01  1.892000e+01  1.866000e+01  1.856000e+01   
     High       1.886000e+01  1.898000e+01  1.884000e+01  1.872000e+01   
     Low        1.856000e+01  1.862000e+01  1.860000e+01  1.846000e+01   
     Close      1.885000e+01  1.872000e+01  1.872000e+01  1.864000e+01   
     Adj Close  1.459206e+01  1.449143e+01  1.449143e+01  1.442949e+01   
...                      ...           ...           ...           ...   
TRV  High       3.746000e+01  3.724000e+01  3.714000e+01  3.756000e+01   
     Low        3.699000e+01  3.688000e+01  3.679000e+01  3.676000e+01   
     Close      3.715000e+01  3.698000e+01  3.707000e+01  3.730000e+01   
     Adj Close  2.576913e+01  2.565120e+01  2.571363e+01  2.587317e+01   
     Volume     2.592200e+06  1.598800e+06  1.528300e+06  2.452600e+06   

Date              2005-01-12    2005-01-13  ...    2019-12-17    2019-12-18  \
CSCO Open       1.876000e+01  1.892000e+01  ...  4.622000e+01  4.652000e+01   
     High       1.897000e+01  1.899000e+01  ...  4.652000e+01  4.676000e+01   
     Low        1.871000e+01  1.863000e+01  ...  4.593000e+01  4.625000e+01   
     Close      1.893000e+01  1.878000e+01  ...  4.644000e+01  4.664000e+01   
     Adj Close  1.465399e+01  1.453787e+01  ...  4.610109e+01  4.629963e+01   
...                      ...           ...  ...           ...           ...   
TRV  High       3.754000e+01  3.765000e+01  ...  1.372600e+02  1.371700e+02   
     Low        3.719000e+01  3.725000e+01  ...  1.362000e+02  1.349400e+02   
     Close      3.754000e+01  3.729000e+01  ...  1.369900e+02  1.356700e+02   
     Adj Close  2.603965e+01  2.586624e+01  ...  1.369900e+02  1.356700e+02   
     Volume     1.753100e+06  1.473700e+06  ...  1.083400e+06  1.582400e+06   

Date              2019-12-19    2019-12-20    2019-12-23     2019-12-24  \
CSCO Open       4.715000e+01  4.825000e+01  4.755000e+01      48.160000   
     High       4.799000e+01  4.837000e+01  4.830000e+01      48.250000   
     Low        4.711000e+01  4.742000e+01  4.742000e+01      47.650002   
     Close      4.788000e+01  4.745000e+01  4.810000e+01      47.779999   
     Adj Close  4.753059e+01  4.710372e+01  4.774898e+01      47.431313   
...                      ...           ...           ...            ...   
TRV  High       1.364900e+02  1.376100e+02  1.368000e+02     136.449997   
     Low        1.351000e+02  1.360400e+02  1.355800e+02     135.550003   
     Close      1.364700e+02  1.364300e+02  1.362300e+02     135.830002   
     Adj Close  1.364700e+02  1.364300e+02  1.362300e+02     135.830002   
     Volume     1.741500e+06  2.676500e+06  1.003000e+06  486200.000000   

Date               2019-12-26     2019-12-27     2019-12-30     2019-12-31  
CSCO Open           47.779999      47.990002      47.750000      47.540001  
     High           48.000000      48.020000      47.880001      47.980000  
     Low            47.650002      47.680000      47.290001      47.410000  
     Close          47.849998      47.770000      47.590000     